In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [2]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc))
# print(inspect.getsource(apps.backward_geocode_slc))

# scene

In [3]:
dem_urlpath = "data/Rome-10m-DEM.tif"

dem_raster = scene.open_dem_raster(dem_urlpath, chunks=2500)
dem_raster

<xarray.DataArray 'band_data' (y: 5000, x: 5000)>
dask.array<getitem, shape=(5000, 5000), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
    spatial_ref  int64 ...

In [4]:
# _ = dem_raster.plot()

In [5]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

CPU times: user 12.6 s, sys: 935 ms, total: 13.5 s
Wall time: 13.3 s


<xarray.DataArray 'dem_3d' (axis: 3, y: 5000, x: 5000)>
dask.array<setitem, shape=(3, 5000, 5000), dtype=float64, chunksize=(3, 2500, 2500), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref  int64 ...
  * y            (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis         (axis) int64 0 1 2

# image

In [6]:
product_urlpath = (
    "data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE/"
)
measurement_group = "IW/VV"
orbit_group = f"{measurement_group}/orbit"
calibration_group = f"{measurement_group}/calibration"
output_urlpath = "Rome-10m-GTC-GRD.tif"

!ls -d {product_urlpath}

data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE/


In [7]:
measurement = xr.open_dataarray(
    product_urlpath,
    engine="sentinel-1",
    group=measurement_group,
    chunks={"pixel": 2048},
)
measurement

<xarray.DataArray 'measurement' (azimuth_time: 16705, ground_range: 26102)>
dask.array<open_dataset-07abf63ac7aa58394121a028c235bfb7measurement, shape=(16705, 26102), dtype=float32, chunksize=(16705, 26102), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(26102,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(16705,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:11:22.594441 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 2.61e+05 2.61e+05
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      1
    long_name:                  digital number

In [8]:
calibration = xr.open_dataset(
    product_urlpath, engine="sentinel-1", group=calibration_group
)
beta_nought_lut = calibration.betaNought
beta_nought_lut

<xarray.DataArray 'betaNought' (line: 27, pixel: 654)>
array([[473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733],
       [473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733],
       [473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733],
       ...,
       [473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733],
       [473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733],
       [473.9733, 473.9733, 473.9733, ..., 473.9733, 473.9733, 473.9733]],
      dtype=float32)
Coordinates:
  * line     (line) int64 0 668 1336 2005 2673 ... 14700 15368 16037 16705 17373
  * pixel    (pixel) int64 0 40 80 120 160 200 ... 25960 26000 26040 26080 26101
Attributes:
    units:      m m-1
    long_name:  beta nought calibration LUT

In [9]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

CPU times: user 3.92 s, sys: 2.03 s, total: 5.95 s
Wall time: 6.01 s


<xarray.DataArray (azimuth_time: 16705, ground_range: 26102)>
dask.array<pow, shape=(16705, 26102), dtype=float32, chunksize=(16705, 26102), chunktype=numpy.ndarray>
Coordinates:
    pixel         (ground_range) int64 dask.array<chunksize=(26102,), meta=np.ndarray>
    line          (azimuth_time) int64 dask.array<chunksize=(16705,), meta=np.ndarray>
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:11:22.594441 .....
  * ground_range  (ground_range) float64 0.0 10.0 20.0 ... 2.61e+05 2.61e+05
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      m2 m-2
    long_name:                  beta nought

In [10]:
orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group)
position_ecef = orbit_ecef.position
position_ecef

<xarray.DataArray 'position' (axis: 3, azimuth_time: 16)>
array([[4657064.97853 , 4712297.916925, 4767003.903681, 4821176.38583 ,
        4874808.871124, 4927894.928873, 4980428.190747, 5032402.351598,
        5083811.170256, 5134648.470349, 5184908.141022, 5234584.137678,
        5283670.482715, 5332161.266259, 5380050.646874, 5427332.852286],
       [1776448.316703, 1777360.958901, 1777994.246465, 1778348.876228,
        1778425.58583 , 1778225.153543, 1777748.398105, 1776996.17854 ,
        1775969.393961, 1774668.983378, 1773095.92551 , 1771251.238588,
        1769135.980159, 1766751.246885, 1764098.174349, 1761177.936816],
       [5013314.106183, 4961243.291607, 4908612.381603, 4855427.302963,
        4801694.045627, 4747418.662004, 4692607.266299, 4637266.033804,
        4581401.200177, 4525019.060727, 4468125.969666, 4410728.339344,
        4352832.639528, 4294445.396682, 4235573.19325 , 4176222.66689 ]])
Coordinates:
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:10:21.029300 .....
  * axis          (axis) int64 0 1 2
Attributes:
    units:      m
    long_name:  ECEF position

In [11]:
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(position_ecef)
position_ecef = orbit_interpolator.position()
velocity_ecef = orbit_interpolator.velocity()
position_ecef

<xarray.DataArray 'position' (azimuth_time: 7501, axis: 3)>
array([[4657064.97861576, 1776448.31671965, 5013314.10621806],
       [4657175.96600368, 1776450.42124249, 5013210.5274712 ],
       [4657286.95131017, 1776452.52464528, 5013106.94646053],
       ...,
       [5427144.94061559, 1761190.14837911, 4176461.01298063],
       [5427238.89771613, 1761184.04313713, 4176341.84088746],
       [5427332.85236519, 1761177.93683131, 4176222.66690769]])
Coordinates:
  * azimuth_time  (azimuth_time) datetime64[ns] 2021-12-23T05:10:21.029300 .....
  * axis          (axis) int64 0 1 2

In [12]:
%%time
dem_coords = geocoding.backward_geocode(dem_ecef, position_ecef, velocity_ecef)
dem_coords

CPU times: user 29.6 s, sys: 10.5 s, total: 40.1 s
Wall time: 26.1 s


<xarray.Dataset>
Dimensions:           (x: 5000, y: 5000, axis: 3)
Coordinates:
  * x                 (x) float64 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref       int64 0
  * y                 (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
  * axis              (axis) int64 0 1 2
Data variables:
    azimuth_time      (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    slant_range_time  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    dem_direction     (axis, y, x) float64 dask.array<chunksize=(3, 2500, 2500), meta=np.ndarray>

In [13]:
coordinate_conversion = xr.open_dataset(
    product_urlpath,
    engine="sentinel-1",
    group=f"{measurement_group}/coordinate_conversion",
)
coordinate_conversion

<xarray.Dataset>
Dimensions:           (azimuth_time: 28, degree: 9)
Coordinates:
  * azimuth_time      (azimuth_time) datetime64[ns] 2021-12-23T05:11:20.68527...
  * degree            (degree) int64 0 1 2 3 4 5 6 7 8
Data variables:
    gr0               (azimuth_time) float64 ...
    sr0               (azimuth_time) float64 ...
    slant_range_time  (azimuth_time) float64 ...
    srgrCoefficients  (azimuth_time, degree) float64 ...
    grsrCoefficients  (azimuth_time, degree) float64 ...
Attributes: (12/15)
    constellation:              sentinel-1
    platform:                   sentinel-1b
    instrument:                 ['c-sar']
    sat:orbit_state:            descending
    sat:absolute_orbit:         30148
    sat:relative_orbit:         22
    ...                         ...
    sar:polarizations:          ['VV', 'VH']
    sar:product_type:           GRD
    xs:instrument_mode_swaths:  ['IW']
    group:                      /IW/VV/coordinate_conversion
    Conventions:                CF-1.8
    history:                    created by xarray_sentinel-0.4.dev15+geff30c0...

In [14]:
slant_range = geocoding.SPEED_OF_LIGHT / 2.0 * dem_coords.slant_range_time
cc = coordinate_conversion.interp(azimuth_time=dem_coords.azimuth_time)
x = slant_range - cc.sr0
ground_range = (cc.srgrCoefficients * x ** cc.degree).sum("degree")
ground_range

<xarray.DataArray (y: 5000, x: 5000)>
dask.array<sum-aggregate, shape=(5000, 5000), dtype=float64, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06

In [15]:
%%time
geocoded = beta_nought.interp(
    azimuth_time=dem_coords.azimuth_time,
    ground_range=ground_range,
    method="linear",
)

geocoded

CPU times: user 6.79 s, sys: 3.86 s, total: 10.6 s
Wall time: 3.2 s


<xarray.DataArray (y: 5000, x: 5000)>
dask.array<dask_aware_interpnd, shape=(5000, 5000), dtype=float32, chunksize=(2500, 2500), chunktype=numpy.ndarray>
Coordinates:
    pixel         (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    line          (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    azimuth_time  (y, x) datetime64[ns] dask.array<chunksize=(2500, 2500), meta=np.ndarray>
    ground_range  (y, x) float64 dask.array<chunksize=(2500, 2500), meta=np.ndarray>
  * x             (x) float64 2.75e+05 2.75e+05 2.75e+05 ... 3.25e+05 3.25e+05
    spatial_ref   int64 0
  * y             (y) float64 4.625e+06 4.625e+06 ... 4.675e+06 4.675e+06
Attributes:
    sar:center_frequency:       5.40500045433435
    sar:pixel_spacing_azimuth:  10.0
    sar:pixel_spacing_range:    10.0
    azimuth_time_interval:      0.00149656999624572
    slant_range_time_interval:  1.554116558005821e-08
    sat:anx_datetime:           2021-12-23T04:33:35.556206Z
    units:                      m2 m-2
    long_name:                  beta nought

In [16]:
# _ = geocoded.plot(vmax=1.0)

In [17]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)